# Introduction to using ParticleSpy

This is an example notebook for performing particle analysis using ParticleSpy.

This notebook takes a HAADF image of nanoparticles, allows the user to determine the best segmentation method and then performs analysis, producing a plot of particle areas.

**Note for M&M 2022**: This notebook was copied from a prior HyperSpy workshop and only minor changes were made to ensure it runs with the current Hyperspy-bundle installation. The original copy can be found at https://github.com/ePSIC-DLS/Hyperspy_Workshop_2022/tree/main/particlespy_demo  - Joshua Taillon, July 25, 2022

Below, we load an image using Hyperspy.

In [ ]:
# Import hyperspy, particlespy, trackpy, and matplotlib
import hyperspy.api as hs
import particlespy.api as ps
import trackpy as tp
import matplotlib.pyplot as plt

In [ ]:
# load data and initialize matplotlib backed 
filename = 'SiO2 HAADF Image.hspy'
haadf = ____

%matplotlib notebook

# plot the HAADF image
____

ParticleSpy works by defining parameters to run segmentation. This is done through the parameters object, which can be loaded with default values.

In [ ]:
# create and generate default segmentation parameters
params = ____()
params.____()

Let's take a look at the default segmentation parameters.

In [ ]:
# examine segmentation parameters
params.____

The default is a simple Otsu threshold with no other options selected. We could apply these parameters to segment the particles but we don't know how well these will work on our individual images. In order to check this out, we can use ParticleSpy's segmentation GUI. Experiment with changing the threshold algorithm and the application of watershedding to obtain the optimum labels. Once happy, click "Get Params" to print the current parameters.

<span style="color:red">WARNING: Don't run this if on Google Cloud Platform!</span>

In [ ]:
# run segmentation UI (will open in new window, so only works when running code locally)
ps.____(____)

# the results stored here provide a decent (but not perfect segmentation)

Next, we use the parameters saved when last pressing Update in the GUI to construct a parameter dictionary.

In [ ]:
# store the parameters generated in the UI
params.____()
params.____['____'] = ____

If running a notebook on a remote server there is an alternative method to check segmentation parameters using the process function (however, this isn't included in the api).

In [ ]:
# run the following to check segmentation parameters if running remotely
import particlespy
seg = particlespy.segptcls.process(haadf,params)
plt.figure()
plt.imshow(seg)

Once we're happy with the segmentation parameters, we run the particle_analysis function to segment our image and provide a list of all the particles.

In [ ]:
# run the actual particle analysis
particles = ps.____(____, ____)

Let's interrogate the particle data a little bit. From an initial image we get a number of particle properties calculated automatically.

In [ ]:
# print the properties of the particles calculated automatically
print(____.list[0].____)

It is then possible to plot a histogram of particle area.

In [ ]:
# plot a histogram of particle sizes
particles.____()

It is also possible to plot a scatter plot of two properties. In this instance, plotting 'intensity' vs area.

In [ ]:
# plot scatter data about particles intensity and size
particles.____(['____','____'])

## ParticleSpy with EDS data

If you have EDS data collected simultaneously with an image, it is possible to extract maps, spectra and the composition from each segmented particle. Firstly, load the EDS spectrum image.

In [ ]:
# load simultaneous EDS data collected with above HAADF
eds_filename = 'SiO2 EDS Spectrum Image.hspy'
eds = ____

Next, put the HAADF image and EDS spectrum image together in a list to make one acquisition.

In [ ]:
# join the HAADF image and EDS spectrum image into a list
# use smaller area (first 32 pixels) to speed up analysis
ac = [haadf.isig[0:32,0:32],eds.inav[0:32,0:32]]

At this point, we need to set the parameters for the EDS analysis. This can be done by taking our previously used parameters object adding parameters with generate_eds().

In [ ]:
# Use generate_eds() method to set parameters for the EDS analysis
params.____(eds_method='CL',elements=['Si','O'], factors=[1.,1.3],store_maps=True)

Now, we need to run ParticleAnalysis again in order to get the accompanying EDS data.

In [ ]:
# analyze particles again to get EDS data with particles
particles2 = ps.____(____, ____)

The extracted maps of each element can be accessed from the particle list from maps['element'].

In [ ]:
# extract Si map and plot
particles2.list[1].____['____'].____()

The composition of each particle can be accessed from the particle list using list[particle].composition.

In [ ]:
# examine the composition of the first particle
particles2.____[0].____

We can plot a radial profile of the particle intensity from both the image and our elemental maps.

In [ ]:
# plot radial profile of particle intensity
rp = ps.____(particles2.____[1], ['____'], plot=True)
plt.show()

In [ ]:
# plot radial profile of elemental maps
rp2 = ps.____(particles2.____[1], ['____','____'], plot=True)
plt.show()

## ParticleSpy with time-series data

ParticleSpy can also be used to track properties through a time-series of images. Let's see that in action on a short series of only 10 frames.

In [ ]:
series = hs.load('particle_series.dm4')

In [ ]:
series.____(navigator='slider')

We'll define some new parameters that work well for this data.

In [ ]:
# create parameters
params_series = ps.____()

In [ ]:
# these settings work well to generate parameters for this data
params_series.____(threshold='mean', min_size=200, gaussian=3)

For particle series, there is a modified ParticleAnalysis function called ParticleAnalysisSeries that deals with multiple frames.

In [ ]:
# do a time-series particle analysis
particles_series = ps.____(____, ____)

In [ ]:
# examine the properties of the particles
particles_series.____[0].____

Now we have a list of particles, with their properties attached including the frame number. We can now pass them through another function that uses the trackpy package to track the same particles through a time series.

In [ ]:
# perform time series analysis
series_test = ps.____(____)

The returned series_test is a pandas dataframe, which is the standard object that is used in trackpy.

We can use trackpy to do cool things like plot the trajectory of all particles in the series. Note: while the axes say they are in pixels, they are actually in nm in this case! (I have just noticed this and will fix)

In [ ]:
# use trackpy to plot particle trajectories
plt.figure()
tp.____(____)

Trackpy also lets you correct for any systematic drift in the series.

In [ ]:
# compute systematic drift
drift = tp.____(____)
drift.plot()

In [ ]:
# remove systematic drift and plot new trajectories
tm = tp.____(____.copy(), ____)
tp.____(____)

Using the particlespy workflow for time-series data, you also have any of the properties specified in the pandas dataframe and you can plot these as a function of frame number.

In [ ]:
# plot particle area vs frame number:
import matplotlib.pyplot as plt
plt.figure()
for index, particle in tm.groupby('particle'):
    plt.plot(particle['frame'], particle['area'], label=index)